## Paquetes y datos originales ## 

In [1251]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list, Main_respiratory_infections_list, LRTI_list, COPD_exacerbations_list, Pulmonary_fibrosis_fibrotorax_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


ImportError: cannot import name 'test_shapiro' from 'auxiliary_functions' (C:\Users\maria\PycharmProjects\practicum\auxiliary_functions.py)

### Previamente he tratado la base de datos, creando nuevas columnas para facilitar la lectura de los datos

In [1252]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


## Dividiré los datos en 3 grupos:
- Con PA (codigo ICD de PA )
- Con PA + MECVV (MECVV positivo + ICD pneumonia < 30 dias)
- Sin PA (ICD pneumonia o ICD pneumonia + MECVV positivo > 30 dias)

# CON PA 

In [1253]:
tabla.PA_diagnosticada.value_counts()

PA_diagnosticada
0.0    33
1.0    17
Name: count, dtype: int64

In [1254]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,4.0,7.4,14.4,80.5,13.95,14.5,6.73,66.0,>60,70
15,75,M,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-09-19 00...",[],...,None,None,None,None,None,None,None,None,None,None
26,97,F,0,"[{'date': '20220419', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'C09BA', 'dataInici': '2022-04-17 00...",[],...,3.2,6.5,12.0,156.9,14.71,5.0,11.37,119.7,52,48
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,3.5,5.8,8.8,186.4,8.31,11.7,2.33,49.9,51,48
29,97,F,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2020-11-30 00...",[],...,None,None,None,None,None,None,None,None,None,None


# CON_PA + MECVV

In [1255]:
CON_PA_MECVV = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30)]
CON_PA_MECVV.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,4.0,7.4,14.4,80.5,13.95,14.5,6.73,66.0,>60,70
27,100,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2021-09-14'}, {'valor...","[{'codi': 'C08CA', 'dataInici': '2020-11-23 00...","[{'data': '20201125131827', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
37,86,M,0,"[{'date': '20221003', 'name': 'GLUCOSA Sèrum',...","[{'imc': '', 'anorexia': '', 'perduaPes': '', ...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '72', 'data': '2022-10-09'}]","[{'codi': 'G04CA', 'dataInici': '2022-09-26 00...","[{'data': '20221012122425', 'disfagia': 'S', '...",...,2.9,5.6,13.5,119.5,8.55,9.5,8.16,114.2,None,46
39,87,F,0,"[{'date': '20230619', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2021-01-17 00...","[{'data': '20220111085147', 'disfagia': 'S', '...",...,None,6.2,11.2,None,22.39,X,14.21,30.7,None,>90
41,93,F,0,"[{'date': '20221013', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '0', 'data': '2023-06-02'}, {'valor...","[{'codi': 'C10AA', 'dataInici': '2021-02-05 00...","[{'data': '20230530155654', 'disfagia': 'S', '...",...,3.5,6.7,13.8,135.7,7.42,27.6,6.43,38.6,>60,48
46,86,M,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],"[{'valor': '55.1', 'data': '2021-03-21'}]","[{'codi': 'G04CA', 'dataInici': '2021-03-20 00...","[{'data': '20210721140136', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
47,94,M,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-03-23 00...","[{'data': '20210325091551', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None


# SIN_PA

In [1256]:
SIN_PA = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) | (tabla["P_diagnosticada"] == 1.0)]
SIN_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...
5,57,M,0,"[{'date': '20220909', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],"[{'valor': '83.7', 'data': '2020-08-13'}, {'va...","[{'codi': 'N02BE', 'dataInici': '2020-08-12 00...",[],...,4.7,6.3,14.9,186,9.45,33.0,None,30.4,>60,>90
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
7,90,F,0,"[{'date': '20220627', 'name': 'GLUCOSA Sèrum',...","[{'imc': '0', 'anorexia': '0', 'perduaPes': '1...","[{'activitat': '3', 'estatMental': '1', 'humit...","[{'dataValoracio': '2020-07-30', 'horaValoraci...",[],"[{'codi': 'C09AA', 'dataInici': '2020-07-31 00...","[{'data': '20200731092102', 'disfagia': 'N', '...",...,None,6.9,13.9,None,7.86,40.6,0.32,38.7,None,78
8,90,F,0,"[{'date': '20220827', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BB', 'dataInici': '2023-07-17 00...",[],...,None,None,13.8,None,5.28,19.1,5.82,35.7,None,74
9,91,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C10AA', 'dataInici': '2020-08-25 00...",[],...,None,None,None,None,None,None,None,None,None,None


Sin aplicar la ultima condicion OR, es decir, queriendo nos devuelva solo las filas que cumplan las 3 condiciones, nos devuelve la: 4, 25, 42, 45 y 48. Sino, nos devuelve todas menos la 43, la cual no tiene un diagnostico de pneumonia.

Ahora que he distribuido los datos en 3 grupos, puedo empezar a analizar su contenido


## Número

In [1257]:
num_CON_PA = len(CON_PA)
num_CON_PA_MECVV= len(CON_PA_MECVV)
num_SIN_PA = len(SIN_PA)

print(num_CON_PA, num_CON_PA_MECVV, num_SIN_PA)

17 10 49


Esto también se puede ver con tan solo mirar las filas que han salido en el apartado anterior, donde vemos que la tabla de CON_PA tiene 17 filas, la de CON_PA_MECVV tiene 10 y la de SIN_PA tiene 49.

#### Pvalor

Para calcular correctamente el p-valor, primero comprobaré si mis datos siguen una distribución normal. Si es así, realizaré un T-test, sino, un test de Mann-Whitney

##### Normalidad

In [1258]:
stat, p_value = shapiro(range(num_CON_PA))
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9621
Valor p = 0.6704
El número de filas sigue una distribución normal (aceptar H0).


In [1259]:
stat, p_value = shapiro(range(num_CON_PA_MECVV))
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9702
Valor p = 0.8924
El número de filas sigue una distribución normal (aceptar H0).


In [1260]:
stat, p_value = shapiro(range(num_SIN_PA))
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9556
Valor p = 0.0626
El número de filas sigue una distribución normal (aceptar H0).


Ahora realizaremos el T-test, teniendo en cuenta que un valor inferior a 0.05 indicará que existen diferencias significativas entre los 2 grupos comparados

In [1261]:
t_stat_12, p_val_12 = ttest_ind(CON_PA.index, CON_PA_MECVV.index, equal_var=False)
print(f"P-valor entre CON_PA y CON_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existen diferencias significativas entre los 2 grupos.")
else:
    print("No existen diferencias significativas entre los 2 grupos.")

P-valor entre CON_PA y CON_PA_MECVV: 0.5246
No existen diferencias significativas entre los 2 grupos.


In [1262]:
t_stat_13, p_val_13 = ttest_ind(CON_PA.index, SIN_PA.index, equal_var=False)
print(f"P-valor entre CON_PA y SIN_PA: {p_val_13:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existen diferencias significativas entre los 2 grupos.")
else:
    print("No existen diferencias significativas entre los 2 grupos.")

P-valor entre CON_PA y SIN_PA: 0.7664
No existen diferencias significativas entre los 2 grupos.


In [1263]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV.index, SIN_PA.index, equal_var=False)
print(f"P-valor entre CON_PA_MECVV y SIN_PA: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existen diferencias significativas entre los 2 grupos.")
else:
    print("No existen diferencias significativas entre los 2 grupos.")

P-valor entre CON_PA_MECVV y SIN_PA: 0.5246
No existen diferencias significativas entre los 2 grupos.


## Edat

In [1264]:
edad_CON_PA = CON_PA['edat'].mean()
desviacion_estandar_edad = CON_PA['edat'].std()

resultado = f"{edad_CON_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)


Promedio ± Desviación Estándar de la Edad: 87.06 ± 8.99


In [1265]:
edad_CON_PA_MECVV = CON_PA_MECVV['edat'].mean()
desviacion_estandar_edad = CON_PA_MECVV['edat'].std()

resultado = f"{edad_CON_PA_MECVV:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)

Promedio ± Desviación Estándar de la Edad: 88.90 ± 8.67


In [1266]:
edad_SIN_PA = SIN_PA['edat'].mean()
desviacion_estandar_edad = SIN_PA['edat'].std()

resultado = f"{edad_SIN_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)

Promedio ± Desviación Estándar de la Edad: 80.12 ± 17.01


#### Pvalor

##### Normalidad 

In [1267]:
stat, p_value = shapiro(CON_PA['edat'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9410
Valor p = 0.3297
El número de filas sigue una distribución normal (aceptar H0).


In [1268]:
stat, p_value = shapiro(CON_PA_MECVV['edat'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9057
Valor p = 0.2528
El número de filas sigue una distribución normal (aceptar H0).


In [1269]:
stat, p_value = shapiro(SIN_PA['edat'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8539
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


Dado que el último grupo (SIN_PA) no sigue una distribución normal, hay que usar el Test de Mann-Whitney en lugar del T-test para realizar las comparaciones

In [1270]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['edat'], CON_PA_MECVV['edat'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.6051223082851969


In [1271]:
stat, p_value_13 = mannwhitneyu(CON_PA['edat'], SIN_PA['edat'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.1815160884632735


In [1272]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['edat'], SIN_PA['edat'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.11458529353557328


## Sexo

In [1273]:
sexe_CON_PA = CON_PA['sexe'].value_counts()
sexe_CON_PA_MECVV = CON_PA_MECVV['sexe'].value_counts()
sexe_SIN_PA = SIN_PA['sexe'].value_counts()

In [1274]:
if 'F' in sexe_CON_PA:
    percent_female1 = (sexe_CON_PA['F'] / len(CON_PA)) * 100
    print(f"Porcentaje de sexo femenino en CON_PA: {percent_female1:.2f}%")
else:
    print("No hay datos de sexo femenino en el CON_PA")

Porcentaje de sexo femenino en CON_PA: 47.06%


In [1275]:
if 'F' in sexe_CON_PA_MECVV:
    percent_female2 = (sexe_CON_PA_MECVV['F'] / len(CON_PA)) * 100
    print(f"Porcentaje de sexo femenino en DataFrame 2: {percent_female2:.2f}%")
else:
    print("No hay datos de sexo femenino en el DataFrame 2.")

Porcentaje de sexo femenino en DataFrame 2: 23.53%


In [1276]:
if 'F' in sexe_SIN_PA:
    percent_female3 = (sexe_SIN_PA['F'] / len(SIN_PA)) * 100
    print(f"Porcentaje de sexo femenino en DataFrame 3: {percent_female3:.2f}%")
else:
    print("No hay datos de sexo femenino en el DataFrame 3.")


Porcentaje de sexo femenino en DataFrame 3: 34.69%


#### Pvalor

##### Normalidad

Al ser una variable categórica no se puede comprobar su normalidad. Realizaré directamente un test chi-cuadrado

In [1277]:
contingency_table_CON_PA_CON_PA_MECVV = pd.crosstab(index=CON_PA['sexe'], columns=CON_PA_MECVV['sexe'])

chi2_CON_PA_CON_PA_MECVV, p_value_CON_PA_CON_PA_MECVV, dof_CON_PA_CON_PA_MECVV, expected_CON_PA_CON_PA_MECVV = chi2_contingency(contingency_table_CON_PA_CON_PA_MECVV)

print("Estadístico Chi-cuadrado entre CON_PA y CON_PA_MECVV:", chi2_CON_PA_CON_PA_MECVV)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_value_CON_PA_CON_PA_MECVV)

if p_value_CON_PA_CON_PA_MECVV < 0.05:
    print("Hay diferencias significativas entre las distribuciones de sexo en CON_PA y CON_PA_MECVV.")
else:
    print("No hay diferencias significativas entre las distribuciones de sexo en CON_PA y CON_PA_MECVV.")

Estadístico Chi-cuadrado entre CON_PA y CON_PA_MECVV: 2.6666666666666665
P-valor entre CON_PA y CON_PA_MECVV: 0.10247043485974942
No hay diferencias significativas entre las distribuciones de sexo en CON_PA y CON_PA_MECVV.


In [1278]:
contingency_table_CON_PA_SIN_PA = pd.crosstab(index=CON_PA['sexe'], columns=SIN_PA['sexe'])

chi2_CON_PA_SIN_PA, p_value_CON_PA_SIN_PA, dof_CON_PA_SIN_PA, expected_CON_PA_SIN_PA = chi2_contingency(contingency_table_CON_PA_SIN_PA)

print("Estadístico Chi-cuadrado entre CON_PA y SIN_PA:", chi2_CON_PA_SIN_PA)
print("P-valor entre CON_PA y SIN_PA:", p_value_CON_PA_SIN_PA)

if p_value_CON_PA_SIN_PA < 0.05:
    print("Hay diferencias significativas entre las distribuciones de sexo en CON_PA y SIN_PA.")
else:
    print("No hay diferencias significativas entre las distribuciones de sexo en CON_PA y SIN_PA.")

Estadístico Chi-cuadrado entre CON_PA y SIN_PA: 13.223042052469136
P-valor entre CON_PA y SIN_PA: 0.00027652843917458906
Hay diferencias significativas entre las distribuciones de sexo en CON_PA y SIN_PA.


In [1279]:
contingency_table_CON_PA_MECVV_SIN_PA = pd.crosstab(index=CON_PA_MECVV['sexe'], columns=SIN_PA['sexe'])

chi2_CON_PA_MECVV_SIN_PA, p_value_CON_PA_MECVV_SIN_PA, dof_CON_PA_MECVV_SIN_PA, expected_CON_PA_MECVV_SIN_PA = chi2_contingency(contingency_table_CON_PA_MECVV_SIN_PA)

print("Estadístico Chi-cuadrado entre CON_PA_MECVV y SIN_PA:", chi2_CON_PA_MECVV_SIN_PA)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_value_CON_PA_MECVV_SIN_PA)

if p_value_CON_PA_MECVV_SIN_PA < 0.05:
    print("Hay diferencias significativas entre las distribuciones de sexo en CON_PA_MECVV y SIN_PA.")
else:
    print("No hay diferencias significativas entre las distribuciones de sexo en CON_PA_MECVV y SIN_PA.")

Estadístico Chi-cuadrado entre CON_PA_MECVV y SIN_PA: 6.267361111111111
P-valor entre CON_PA_MECVV y SIN_PA: 0.012298217061234292
Hay diferencias significativas entre las distribuciones de sexo en CON_PA_MECVV y SIN_PA.


## Ingressos (admissions)

In [1280]:
promedio_ingresos = CON_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 5.59 ± 3.37


In [1281]:
promedio_ingresos = CON_PA_MECVV['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA_MECVV['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 5.90 ± 2.08


In [1282]:
promedio_ingresos = SIN_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = SIN_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 6.69 ± 5.13


#### Pvalor

##### Normalidad

In [1283]:
stat, p_value = shapiro(CON_PA['Num_ingresos'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8008
Valor p = 0.0021
El número de filas no sigue una distribución normal (rechazar H0).


In [1284]:
stat, p_value = shapiro(CON_PA_MECVV['Num_ingresos'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9558
Valor p = 0.7376
El número de filas sigue una distribución normal (aceptar H0).


In [1285]:
stat, p_value = shapiro(SIN_PA['Num_ingresos'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.7277
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


Tendré que hacer test de Mann-Whitney en las 3 comparaciones ya que 2 de los 3 grupos no siguen una distribución normal.

In [1286]:
stat, p_value_12 = mannwhitneyu(CON_PA['Num_ingresos'], CON_PA_MECVV['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.3599581940859352


In [1287]:
stat, p_value_13 = mannwhitneyu(CON_PA['Num_ingresos'], SIN_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.514931142191354


In [1288]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['Num_ingresos'], SIN_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.7831397976578486


## Disfàgia

In [1289]:
disf_CON_PA= (CON_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_CON_PA:.2f}%")

Porcentaje de disfagia: 47.06%


In [1290]:
disf_CON_PA_MECVV = (CON_PA_MECVV['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_CON_PA_MECVV:.2f}%")

Porcentaje de disfagia: 30.00%


In [1291]:
disf_SIN_PA= (SIN_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_SIN_PA:.2f}%")

Porcentaje de disfagia: 18.37%


#### Pvalor

##### Normalidad

In [1292]:
stat, p_value = shapiro(CON_PA['DO_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6418
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [1293]:
stat, p_value = shapiro(CON_PA_MECVV['DO_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5942
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [1294]:
stat, p_value = shapiro(SIN_PA['DO_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


Ningun grupo sigue una distribución normal, y como previamente he transformado esta variable categorica (NO, SI) en una binaria (0,1), puedo aplicar el test de Mann-Whitney

In [1295]:
stat, p_value_12 = mannwhitneyu(CON_PA['DO_diagnosticada'], CON_PA_MECVV['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.40914513229166993


In [1296]:
stat, p_value_13 = mannwhitneyu(CON_PA['DO_diagnosticada'], SIN_PA['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.02125060274444335


In [1297]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['DO_diagnosticada'], SIN_PA['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.4172116903428992


## Index de Barthel

### Promedio

In [1298]:
promedio_barthel = CON_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 22.65 ± 24.63


In [1299]:
promedio_barthel = CON_PA_MECVV['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA_MECVV['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 28.50 ± 22.24


In [1300]:
promedio_barthel = SIN_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = SIN_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 48.88 ± 37.42


#### Pvalor

##### Normalidad

In [1301]:
stat, p_value = shapiro(CON_PA['Barthel_resultados'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8517
Valor p = 0.0114
El número de filas no sigue una distribución normal (rechazar H0).


In [1302]:
stat, p_value = shapiro(CON_PA_MECVV['Barthel_resultados'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9138
Valor p = 0.3083
El número de filas sigue una distribución normal (aceptar H0).


In [1303]:
stat, p_value = shapiro(SIN_PA['Barthel_resultados'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8858
Valor p = 0.0002
El número de filas no sigue una distribución normal (rechazar H0).


Como la mayoria no sigue una distribucion normal, hay que usar el test Mann-Whitney

In [1304]:
stat, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'], CON_PA_MECVV['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.41631085084208264


In [1305]:
stat, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'], SIN_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.012373091641540783


In [1306]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'], SIN_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.13803563369050034


### Valores de dependencia

##### CON_PA

In [1307]:
indepe_CON_PA = (CON_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_CON_PA: {indepe_CON_PA:.2f}%")

Porcentaje de indepe_CON_PA: 0.00%


In [1308]:
mod_CON_PA = (CON_PA['Barthel_resultados'].between(61, 95)).mean() * 100
print(f"Porcentaje de mod_CON_PA: {mod_CON_PA:.2f}%")

Porcentaje de mod_CON_PA: 5.88%


In [1309]:
sev_CON_PA = (CON_PA['Barthel_resultados'].between(21, 60)).mean() * 100
print(f"Porcentaje de sev_CON_PA: {sev_CON_PA:.2f}%")

Porcentaje de sev_CON_PA: 41.18%


In [1310]:
total_CON_PA = (CON_PA['Barthel_resultados'] <= 20.0 ).mean() * 100
print(f"Porcentaje de total_CON_PA: {total_CON_PA:.2f}%")

Porcentaje de total_CON_PA: 52.94%


##### CON_PA_MECVV

In [1311]:
indepe_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_CON_PA_MECVV: {indepe_CON_PA_MECVV:.2f}%")

Porcentaje de indepe_CON_PA_MECVV: 0.00%


In [1312]:
mod_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Porcentaje de mod_CON_PA_MECVV: {mod_CON_PA_MECVV:.2f}%")

Porcentaje de mod_CON_PA_MECVV: 0.00%


In [1313]:
sev_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Porcentaje de sev_CON_PA_MECVV: {sev_CON_PA_MECVV:.2f}%")

Porcentaje de sev_CON_PA_MECVV: 60.00%


In [1314]:
total_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] <= 20.0).mean() * 100
print(f"Porcentaje de total_CON_PA_MECVV: {total_CON_PA_MECVV:.2f}%")

Porcentaje de total_CON_PA_MECVV: 40.00%


##### SIN_PA

In [1315]:
indepe_SIN_PA = (SIN_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_SIN_PA: {indepe_SIN_PA:.2f}%")

Porcentaje de indepe_SIN_PA: 20.41%


In [1316]:
mod_SIN_PA = (SIN_PA['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Porcentaje de mod_SIN_PA: {mod_SIN_PA:.2f}%")

Porcentaje de mod_SIN_PA: 18.37%


In [1317]:
sev_SIN_PA = (SIN_PA['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Porcentaje de sev_SIN_PA: {sev_SIN_PA:.2f}%")

Porcentaje de sev_SIN_PA: 28.57%


In [1318]:
total_SIN_PA = (SIN_PA['Barthel_resultados'] <= 20).mean() * 100
print(f"Porcentaje de total_SIN_PA: {total_SIN_PA:.2f}%")

Porcentaje de total_SIN_PA: 32.65%


#### Pvalor

##### Independencia

In [1319]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'] == 100.0, CON_PA_MECVV['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_CON_PA y indepe_CON_PA_MECVV:", p_value_12)

P-valor entre indepe_CON_PA y indepe_CON_PA_MECVV: 1.0


In [1320]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_CON_PA y indepe_SIN_PA:", p_value_13)

P-valor entre indepe_CON_PA y indepe_SIN_PA: 0.04605091215969595


In [1321]:
stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA_MECVV y SIN_PA:", p_value_23)

P-valor entre indepe_CON_PA_MECVV y SIN_PA: 0.13803563369050034


##### Moderada

In [1322]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'].between(61, 95), CON_PA_MECVV['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_CON_PA y mod_CON_PA_MECVV:", p_value_12)

P-valor entre mod_CON_PA y mod_CON_PA_MECVV: 0.49002535950179893


In [1323]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre indepe_CON_PA y mod_SIN_PA:", p_value_13)

P-valor entre indepe_CON_PA y mod_SIN_PA: 0.22404485220846948


In [1324]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')  
print("P-valor entre mod_CON_PA_MECVV y mod_SIN_PA :", p_value_23)

P-valor entre mod_CON_PA_MECVV y mod_SIN_PA : 0.14889565499123467


##### Severa

In [1325]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'].between(21,60), CON_PA_MECVV['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_CON_PA y sev_CON_PA_MECVV:", p_value_12)

P-valor entre sev_CON_PA y sev_CON_PA_MECVV: 0.3688417252786701


In [1326]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_CON_PA y sev_SIN_PA:", p_value_13)

P-valor entre sev_CON_PA y sev_SIN_PA: 0.3446368758082651


In [1327]:
t_stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), alternative='two-sided' )
print("P-valor entre sev_CON_PA_MECVV y sev_SIN_PA:", p_value_23)

P-valor entre sev_CON_PA_MECVV y sev_SIN_PA: 0.05945988532601827


##### Total

In [1328]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'] <= 20.0, CON_PA_MECVV['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA y total_CON_PA_MECVV:", p_value_12)

P-valor entre total_CON_PA y total_CON_PA_MECVV: 0.5426838040005246


In [1329]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA y total_SIN_PA:", p_value_13)

P-valor entre total_CON_PA y total_SIN_PA: 0.14269651822055499


In [1330]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA_MECVV y total_SIN_PA:", p_value_23)

P-valor entre total_CON_PA_MECVV y total_SIN_PA: 0.6663533091466043


## Index del MNA

Primero hay que convertir la columna a tipo numerico ya que es de tipo "object" y no permite realizar el promedio

In [1331]:
CON_PA.loc[:, 'mna_resultats'] = pd.to_numeric(CON_PA['mna_resultats'], errors='coerce')
CON_PA_MECVV.loc[:, 'mna_resultats'] = pd.to_numeric(CON_PA_MECVV['mna_resultats'], errors='coerce')
SIN_PA.loc[:, 'mna_resultats'] = pd.to_numeric(SIN_PA['mna_resultats'], errors='coerce')

### Promedio

In [1332]:
promedio_mna = CON_PA['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 0.00 ± 0.00


In [1333]:
promedio_mna = CON_PA_MECVV['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA_MECVV['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 0.00 ± nan


In [1334]:
promedio_mna = SIN_PA['mna_resultats'].mean()
desviacion_estandar_mna = SIN_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 3.05 ± 5.92


#### Pvalor

##### Normalidad

In [1500]:
conpanone = CON_PA[CON_PA['mna_resultats'].notna()]
stat, p_value = shapiro(conpanone)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = nan
Valor p = nan
El número de filas sigue una distribución normal (aceptar H0).


In [1501]:
pa2none = CON_PA_MECVV[CON_PA_MECVV['mna_resultats'].notna()]
stat, p_value = shapiro(pa2none)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = nan
Valor p = nan
El número de filas sigue una distribución normal (aceptar H0).


In [1502]:
pa3none = SIN_PA[SIN_PA['mna_resultats'].notna()]
stat, p_value = shapiro(pa3none)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = nan
Valor p = nan
El número de filas sigue una distribución normal (aceptar H0).


In [1339]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'], CON_PA_MECVV['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1340]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1341]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Valores de malnutricion

##### CON_PA

In [1342]:
norm_CON_PA = CON_PA['mna_resultats'].between(24, 30)
porcentaje_norm_CON_PA = norm_CON_PA.mean() * 100
num_observaciones_norm_CON_PA = norm_CON_PA.sum()

print(f"Porcentaje de norm_CON_PA: {porcentaje_norm_CON_PA:.2f}%")
print(f"Número de observaciones en norm_CON_PA: {num_observaciones_norm_CON_PA}")

Porcentaje de norm_CON_PA: 0.00%
Número de observaciones en norm_CON_PA: 0


In [1343]:
malnutr_CON_PA = CON_PA['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_CON_PA = malnutr_CON_PA.mean() * 100
num_observaciones_malnutr_CON_PA = norm_CON_PA.sum()

print(f"Porcentaje de malnutr_CON_PA: {porcentaje_malnutr_CON_PA:.2f}%")
print(f"Número de observaciones en norm_CON_PA: {num_observaciones_malnutr_CON_PA}")

Porcentaje de malnutr_CON_PA: 0.00%
Número de observaciones en norm_CON_PA: 0


In [1344]:
malnut_CON_PA = CON_PA['mna_resultats'] < 17
porcentaje_malnut_CON_PA = malnut_CON_PA.mean() * 100
num_observaciones_malnut_CON_PA = malnut_CON_PA.sum()

print(f"Porcentaje de malnut_CON_PA: {porcentaje_malnut_CON_PA:.2f}%")
print(f"Número de observaciones en malnut_CON_PA: {num_observaciones_malnut_CON_PA}")

Porcentaje de malnut_CON_PA: 23.53%
Número de observaciones en malnut_CON_PA: 4


##### CON_PA_MECVV

In [1345]:
norm_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(24, 30)
porcentaje_norm_CON_PA_MECVV = norm_CON_PA_MECVV.mean() * 100
num_observaciones_norm_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Porcentaje de norm_CON_PA_MECVV: {porcentaje_norm_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en norm_CON_PA_MECVV: {num_observaciones_norm_CON_PA_MECVV}")

Porcentaje de norm_CON_PA_MECVV: 0.00%
Número de observaciones en norm_CON_PA_MECVV: 0


In [1346]:
malnutr_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_CON_PA_MECVV = malnutr_CON_PA_MECVV.mean() * 100
num_observaciones_malnutr_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Porcentaje de malnutr_CON_PA_MECVV: {porcentaje_malnutr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en norm_CON_PA_MECVV: {num_observaciones_malnutr_CON_PA_MECVV}")

Porcentaje de malnutr_CON_PA_MECVV: 0.00%
Número de observaciones en norm_CON_PA_MECVV: 0


In [1347]:
malnut_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'] < 17
porcentaje_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.mean() * 100
num_observaciones_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.sum()

print(f"Porcentaje de malnut_CON_PA_MECVV: {porcentaje_malnut_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en malnut_CON_PA_MECVV: {num_observaciones_malnut_CON_PA_MECVV}")

Porcentaje de malnut_CON_PA_MECVV: 10.00%
Número de observaciones en malnut_CON_PA_MECVV: 1


##### SIN_PA

In [1348]:
norm_SIN_PA = SIN_PA['mna_resultats'].between(24, 30)
porcentaje_norm_SIN_PA = norm_SIN_PA.mean() * 100
num_observaciones_norm_SIN_PA = norm_SIN_PA.sum()

print(f"Porcentaje de norm_SIN_PA: {porcentaje_norm_SIN_PA:.2f}%")
print(f"Número de observaciones en norm_SIN_PA: {num_observaciones_norm_SIN_PA}")

Porcentaje de norm_SIN_PA: 0.00%
Número de observaciones en norm_SIN_PA: 0


In [1349]:
malnutr_SIN_PA = SIN_PA['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_SIN_PA = malnutr_SIN_PA.mean() * 100
num_observaciones_malnutr_SIN_PA = malnutr_SIN_PA.sum()

print(f"Porcentaje de malnutr_SIN_PA: {porcentaje_malnutr_SIN_PA:.2f}%")
print(f"Número de observaciones en malnutr_SIN_PA: {num_observaciones_malnutr_SIN_PA}")

Porcentaje de malnutr_SIN_PA: 2.04%
Número de observaciones en malnutr_SIN_PA: 1


In [1350]:
malnut_SIN_PA = SIN_PA['mna_resultats'] < 17
porcentaje_malnut_SIN_PA = malnut_SIN_PA.mean() * 100
num_observaciones_malnut_SIN_PA = malnut_SIN_PA.sum()

print(f"Porcentaje de malnut_SIN_PA: {porcentaje_malnut_SIN_PA:.2f}%")
print(f"Número de observaciones en malnut_SIN_PA: {num_observaciones_malnut_SIN_PA}")

Porcentaje de malnut_SIN_PA: 18.37%
Número de observaciones en malnut_SIN_PA: 9


#### Pvalor

##### Normal

In [1504]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'].between(24, 30), CON_PA_MECVV['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA y norm_CON_PA_MECVV:", p_val_12)

P-valor entre norm_CON_PA y norm_CON_PA_MECVV: 1.0


In [1505]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA y norm_SIN_PA:", p_val_13)

P-valor entre norm_CON_PA y norm_SIN_PA: 1.0


In [1506]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA_MECVV y norm_SIN_PA:", p_val_23)

P-valor entre norm_CON_PA_MECVV y norm_SIN_PA: 1.0


##### Riesgo de malnutricion

In [1507]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'].between(17, 23.5), CON_PA_MECVV['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA y malnutr_CON_PA_MECVV:", p_val_12)

P-valor entre malnutr_CON_PA y malnutr_CON_PA_MECVV: 1.0


In [1508]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA y malnutr_SIN_PA:", p_val_13)

P-valor entre malnutr_CON_PA y malnutr_SIN_PA: 0.5793276078637376


In [1509]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA_MECVV y malnutr_SIN_PA:", p_val_23)

P-valor entre malnutr_CON_PA_MECVV y malnutr_SIN_PA: 0.6843175494727258


##### Malnutricion

In [1514]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'] <17 , 
                                CON_PA_MECVV['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA y norm_CON_PA_MECVV:", p_val_12)

P-valor entre malnut_CON_PA y norm_CON_PA_MECVV: 3.0070483450109954e-06


In [1512]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA y malnut_SIN_PA:", p_val_13)

P-valor entre malnut_CON_PA y malnut_SIN_PA: 4.065922020960315e-09


In [1515]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA_MECVV y malnut_SIN_PA:", p_val_23)

P-valor entre malnut_CON_PA_MECVV y malnut_SIN_PA: 0.5341419377112933


## Indice de la  EMINA

### Promedio

In [1360]:
promedio_emina = CON_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = CON_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Promedio ± Desviación Estándar de la emina:", resultado)

Promedio ± Desviación Estándar de la emina: 7.94 ± 3.21


In [1361]:
promedio_emina = CON_PA_MECVV['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = CON_PA_MECVV['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Promedio ± Desviación Estándar de la emina:", resultado)

Promedio ± Desviación Estándar de la emina: 7.60 ± 2.01


In [1362]:
promedio_emina = SIN_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = SIN_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Promedio ± Desviación Estándar de la emina:", resultado)

Promedio ± Desviación Estándar de la emina: 5.64 ± 4.08


#### Pvalor

##### Normalidad

In [1524]:
stat, p_value = shapiro(CON_PA['EMINA_sumatorios_comparados'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9770
Valor p = 0.9248
El número de filas sigue una distribución normal (aceptar H0).


In [1523]:
stat, p_value = shapiro(CON_PA_MECVV['EMINA_sumatorios_comparados'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8885
Valor p = 0.1630
El número de filas sigue una distribución normal (aceptar H0).


In [1522]:
stat, p_value = shapiro(SIN_PA['EMINA_sumatorios_comparados'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = nan
Valor p = nan
El número de filas sigue una distribución normal (aceptar H0).


Podemos usar T-test porque siguen distribucion normal (?)

In [1525]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'], CON_PA_MECVV['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.7372065144531302


In [1364]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'], SIN_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1365]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'], SIN_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Valores EMINA

##### CON_PA

In [1366]:
lowr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(0, 5)
porcentaje_lowr_CON_PA = lowr_CON_PA.mean() * 100
num_observaciones_lowr_CON_PA = lowr_CON_PA.sum()

print(f"Porcentaje de lowr_CON_PA: {porcentaje_lowr_CON_PA:.2f}%")
print(f"Número de observaciones en lowr_CON_PA: {num_observaciones_lowr_CON_PA}")

Porcentaje de lowr_CON_PA: 17.65%
Número de observaciones en lowr_CON_PA: 3


In [1367]:
modr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(6, 10)
porcentaje_modr_CON_PA = modr_CON_PA.mean() * 100
num_observaciones_modr_CON_PA = modr_CON_PA.sum()

print(f"Porcentaje de modr_CON_PA: {porcentaje_modr_CON_PA:.2f}%")
print(f"Número de observaciones en modr_CON_PA: {num_observaciones_modr_CON_PA}")

Porcentaje de modr_CON_PA: 64.71%
Número de observaciones en modr_CON_PA: 11


In [1368]:
highr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(11, 15)
porcentaje_highr_CON_PA = highr_CON_PA.mean() * 100
num_observaciones_highr_CON_PA = highr_CON_PA.sum()

print(f"Porcentaje de highr_CON_PA: {porcentaje_highr_CON_PA:.2f}%")
print(f"Número de observaciones en highr_CON_PA: {num_observaciones_highr_CON_PA}")

Porcentaje de highr_CON_PA: 17.65%
Número de observaciones en highr_CON_PA: 3


##### CON_PA_MECVV

In [1369]:
lowr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0, 5)
porcentaje_lowr_CON_PA_MECVV = lowr_CON_PA_MECVV.mean() * 100
num_observaciones_lowr_CON_PA_MECVV = lowr_CON_PA_MECVV.sum()

print(f"Porcentaje de lowr_CON_PA_MECVV: {porcentaje_lowr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en lowr_CON_PA_MECVV: {num_observaciones_lowr_CON_PA_MECVV}")

Porcentaje de lowr_CON_PA_MECVV: 20.00%
Número de observaciones en lowr_CON_PA_MECVV: 2


In [1370]:
modr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6, 10)
porcentaje_modr_CON_PA_MECVV = modr_CON_PA_MECVV.mean() * 100
num_observaciones_modr_CON_PA_MECVV = modr_CON_PA_MECVV.sum()

print(f"Porcentaje de modr_CON_PA_MECVV: {porcentaje_modr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en modr_CON_PA_MECVV: {num_observaciones_modr_CON_PA_MECVV}")

Porcentaje de modr_CON_PA_MECVV: 80.00%
Número de observaciones en modr_CON_PA_MECVV: 8


In [1371]:
highr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11, 15)
porcentaje_highr_CON_PA_MECVV = highr_CON_PA_MECVV.mean() * 100
num_observaciones_highr_CON_PA_MECVV = highr_CON_PA_MECVV.sum()

print(f"Porcentaje de highr_CON_PA_MECVV: {porcentaje_highr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en highr_CON_PA_MECVV: {num_observaciones_highr_CON_PA_MECVV}")

Porcentaje de highr_CON_PA_MECVV: 0.00%
Número de observaciones en highr_CON_PA_MECVV: 0


##### SIN_PA

In [1372]:
lowr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(0, 5)
porcentaje_lowr_SIN_PA = lowr_SIN_PA.mean() * 100
num_observaciones_lowr_SIN_PA = lowr_SIN_PA.sum()

print(f"Porcentaje de lowr_SIN_PA: {porcentaje_lowr_SIN_PA:.2f}%")
print(f"Número de observaciones en lowr_SIN_PA: {num_observaciones_lowr_SIN_PA}")

Porcentaje de lowr_SIN_PA: 44.90%
Número de observaciones en lowr_SIN_PA: 22


In [1373]:
modr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(6, 10)
porcentaje_modr_SIN_PA = modr_SIN_PA.mean() * 100
num_observaciones_modr_SIN_PA = modr_SIN_PA.sum()

print(f"Porcentaje de modr_SIN_PA: {porcentaje_modr_SIN_PA:.2f}%")
print(f"Número de observaciones en modr_SIN_PA: {num_observaciones_modr_SIN_PA}")

Porcentaje de modr_SIN_PA: 40.82%
Número de observaciones en modr_SIN_PA: 20


In [1374]:
highr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(11, 15)
porcentaje_highr_SIN_PA = highr_SIN_PA.mean() * 100
num_observaciones_highr_SIN_PA = highr_SIN_PA.sum()

print(f"Porcentaje de highr_SIN_PA: {porcentaje_highr_SIN_PA:.2f}%")
print(f"Número de observaciones en highr_SIN_PA: {num_observaciones_highr_SIN_PA}")

Porcentaje de highr_SIN_PA: 10.20%
Número de observaciones en highr_SIN_PA: 5


#### Pvalor 

##### Riesgo bajo

In [1375]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(0,5), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.8874457485314164


In [1376]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(0,5), SIN_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.028470032037651503


In [1377]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), SIN_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.12129243251471264


##### Riesgo moderado 

In [1378]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(6,10), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.40237952012988454


In [1379]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(6,10), SIN_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.09654829012396034


In [1380]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), SIN_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.02066325652755892


##### Riesgo alto

In [1381]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(11,15), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.08262239198015026


In [1382]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(11,15), SIN_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.48486212369214565


In [1383]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), SIN_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.02374418239053046


## Indice de la canadenca 

### Promedio

In [1384]:
promedio_canad = CON_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = CON_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Promedio ± Desviación Estándar de la canadenca:", resultado)

Promedio ± Desviación Estándar de la canadenca: 7.00 ± nan


In [1385]:
promedio_canad = CON_PA_MECVV['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = CON_PA_MECVV['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Promedio ± Desviación Estándar de la canadenca:", resultado)

Promedio ± Desviación Estándar de la canadenca: 7.00 ± nan


In [1386]:
promedio_canad = SIN_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = SIN_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Promedio ± Desviación Estándar de la canadenca:", resultado)

Promedio ± Desviación Estándar de la canadenca: 8.00 ± 1.41


#### Pvalor

##### Normalidad

In [1532]:
No puedo calcular la normalidad porque solo hay 2 filas que tienen un valor, el resto estan vacias. Asumo que no sigue una distribución normal.

ValueError: Data must be at least length 3.

In [1533]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'], CON_PA_MECVV['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1534]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'], SIN_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1535]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'], SIN_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Valores canadenca

##### CON_PA 

In [1390]:
mild_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
porcentaje_mild_CON_PA = mild_CON_PA.mean() * 100
num_observaciones_mild_CON_PA = mild_CON_PA.sum()

print(f"Porcentaje de mild_CON_PA: {porcentaje_mild_CON_PA:.2f}%")
print(f"Número de observaciones en mild_CON_PA: {num_observaciones_mild_CON_PA}")

Porcentaje de mild_CON_PA: 0.00%
Número de observaciones en mild_CON_PA: 0


In [1391]:
modneu_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(5,10)
porcentaje_modneu_CON_PA = modneu_CON_PA.mean() * 100
num_observaciones_modneu_CON_PA = modneu_CON_PA.sum()

print(f"Porcentaje de modneu_CON_PA: {porcentaje_modneu_CON_PA:.2f}%")
print(f"Número de observaciones en modneu_CON_PA: {num_observaciones_modneu_CON_PA}")

Porcentaje de modneu_CON_PA: 5.88%
Número de observaciones en modneu_CON_PA: 1


In [1392]:
sevneu_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
porcentaje_sevneu_CON_PA = sevneu_CON_PA.mean() * 100
num_observaciones_sevneu_CON_PA = sevneu_CON_PA.sum()

print(f"Porcentaje de sevneu_CON_PA: {porcentaje_sevneu_CON_PA:.2f}%")
print(f"Número de observaciones en sevneu_CON_PA: {num_observaciones_sevneu_CON_PA}")

Porcentaje de sevneu_CON_PA: 0.00%
Número de observaciones en sevneu_CON_PA: 0


##### CON_PA_MECVV

In [1393]:
mild_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
porcentaje_mild_CON_PA_MECVV = mild_CON_PA_MECVV.mean() * 100
num_observaciones_mild_CON_PA_MECVV = mild_CON_PA_MECVV.sum()

print(f"Porcentaje de mild_CON_PA_MECVV: {porcentaje_mild_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en mild_CON_PA_MECVV: {num_observaciones_mild_CON_PA_MECVV}")

Porcentaje de mild_CON_PA_MECVV: 0.00%
Número de observaciones en mild_CON_PA_MECVV: 0


In [1394]:
modneu_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10)
porcentaje_modneu_CON_PA_MECVV = modneu_CON_PA_MECVV.mean() * 100
num_observaciones_modneu_CON_PA_MECVV = modneu_CON_PA_MECVV.sum()

print(f"Porcentaje de modneu_CON_PA_MECVV: {porcentaje_modneu_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en modneu_CON_PA_MECVV: {num_observaciones_modneu_CON_PA_MECVV}")

Porcentaje de modneu_CON_PA_MECVV: 10.00%
Número de observaciones en modneu_CON_PA_MECVV: 1


In [1395]:
sevneu_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
porcentaje_sevneu_CON_PA_MECVV = sevneu_CON_PA_MECVV.mean() * 100
num_observaciones_sevneu_CON_PA_MECVV = sevneu_CON_PA_MECVV.sum()

print(f"Porcentaje de sevneu_CON_PA_MECVV: {porcentaje_sevneu_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en sevneu_CON_PA_MECVV: {num_observaciones_sevneu_CON_PA_MECVV}")

Porcentaje de sevneu_CON_PA_MECVV: 0.00%
Número de observaciones en sevneu_CON_PA_MECVV: 0


##### SIN_PA

In [1396]:
mild_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
porcentaje_mild_SIN_PA = mild_SIN_PA.mean() * 100
num_observaciones_mild_SIN_PA = mild_SIN_PA.sum()

print(f"Porcentaje de mild_SIN_PA: {porcentaje_mild_SIN_PA:.2f}%")
print(f"Número de observaciones en mild_SIN_PA: {num_observaciones_mild_SIN_PA}")

Porcentaje de mild_SIN_PA: 0.00%
Número de observaciones en mild_SIN_PA: 0


In [1397]:
modneu_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(5,10)
porcentaje_modneu_SIN_PA = modneu_SIN_PA.mean() * 100
num_observaciones_modneu_SIN_PA = modneu_SIN_PA.sum()

print(f"Porcentaje de modneu_SIN_PA: {porcentaje_modneu_SIN_PA:.2f}%")
print(f"Número de observaciones en modneu_SIN_PA: {num_observaciones_modneu_SIN_PA}")

Porcentaje de modneu_SIN_PA: 4.08%
Número de observaciones en modneu_SIN_PA: 2


In [1398]:
sevneu_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
porcentaje_sevneu_SIN_PA = sevneu_SIN_PA.mean() * 100
num_observaciones_sevneu_SIN_PA = sevneu_SIN_PA.sum()

print(f"Porcentaje de sevneu_SIN_PA: {porcentaje_sevneu_SIN_PA:.2f}%")
print(f"Número de observaciones en sevneu_SIN_PA: {num_observaciones_sevneu_SIN_PA}")

Porcentaje de sevneu_SIN_PA: 0.00%
Número de observaciones en sevneu_SIN_PA: 0


#### Pvalor

##### Deficit neurologico ligero (mild)

In [1399]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1400]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), SIN_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1401]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), SIN_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


##### Deficit neurologico moderado

In [1536]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'].between(5,10), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.3700051180291794


In [1539]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'].between(5,10), SIN_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.6274490868971419


In [1538]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10), SIN_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.7870742763184819


##### Deficit neurologico severo

In [1405]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(1.5,4.5), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1406]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1407]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


## Infecciones respiratorias

##### CON_PA 

In [1408]:
mainresp_CON_PA= (CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de infecciones respiratorias principales : {mainresp_CON_PA:.2f}%")

Porcentaje de infecciones respiratorias principales : 41.18%


In [1409]:
LRTI_CON_PA= (CON_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de LRTI: {LRTI_CON_PA:.2f}%")

Porcentaje de LRTI: 17.65%


In [1410]:
COPD_CON_PA= (CON_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de exacerbaciones de COPD: {COPD_CON_PA:.2f}%")

Porcentaje de exacerbaciones de COPD: 29.41%


In [1411]:
fibro_CON_PA= (CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de fibrosis pulmonar y fibrotorax: {fibro_CON_PA:.2f}%")

Porcentaje de fibrosis pulmonar y fibrotorax: 0.00%


##### CON_PA_MECVV

In [1412]:
mainresp_CON_PA_MECVV= (CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de infecciones respiratorias principales : {mainresp_CON_PA_MECVV:.2f}%")

Porcentaje de infecciones respiratorias principales : 50.00%


In [1413]:
LRTI_CON_PA_MECVV= (CON_PA_MECVV['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de LRTI: {LRTI_CON_PA_MECVV:.2f}%")

Porcentaje de LRTI: 20.00%


In [1414]:
COPD_CON_PA_MECVV = (CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de exacerbaciones de COPD: {COPD_CON_PA_MECVV:.2f}%")

Porcentaje de exacerbaciones de COPD: 30.00%


In [1415]:
fibro_CON_PA_MECVV = (CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de fibrosis pulmonar y fibrotorax: {fibro_CON_PA_MECVV:.2f}%")

Porcentaje de fibrosis pulmonar y fibrotorax: 0.00%


##### SIN_PA

In [1416]:
mainresp_SIN_PA= (SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de infecciones respiratorias principales : {mainresp_SIN_PA:.2f}%")

Porcentaje de infecciones respiratorias principales : 42.86%


In [1417]:
LRTI_SIN_PA= (SIN_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de LRTI: {LRTI_SIN_PA:.2f}%")

Porcentaje de LRTI: 22.45%


In [1418]:
COPD_SIN_PA= (SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de exacerbaciones de COPD: {COPD_SIN_PA:.2f}%")

Porcentaje de exacerbaciones de COPD: 32.65%


In [1419]:
fibro_SIN_PA= (SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de fibrosis pulmonar y fibrotorax: {fibro_SIN_PA:.2f}%")

Porcentaje de fibrosis pulmonar y fibrotorax: 0.00%


#### Pvalor

##### Normalidad

In [1554]:
stat, p_value = shapiro(CON_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6317
Valor p = 0.00002144
El número de filas no sigue una distribución normal (rechazar H0).


In [1553]:
stat, p_value = shapiro(CON_PA['LRTI_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.4702
Valor p = 0.00000073
El número de filas no sigue una distribución normal (rechazar H0).


In [1551]:
stat, p_value = shapiro(CON_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5785
Valor p = 0.00000650
El número de filas no sigue una distribución normal (rechazar H0).


In [1550]:
stat, p_value = shapiro(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.00000000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Usaré Mann-Whitney cuando tenga que comparar contra CON_PA

In [1549]:
stat, p_value = shapiro(CON_PA_MECVV['Main_respiratory_infections_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6553
Valor p = 0.00025396
El número de filas no sigue una distribución normal (rechazar H0).


In [1548]:
stat, p_value = shapiro(CON_PA_MECVV['LRTI_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5093
Valor p = 0.00000467
El número de filas no sigue una distribución normal (rechazar H0).


In [1547]:
stat, p_value = shapiro(CON_PA_MECVV['COPD_exacerbations_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5942
Valor p = 0.00004713
El número de filas no sigue una distribución normal (rechazar H0).


In [1556]:
stat, p_value = shapiro(CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.00000000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Usaré también Mann-Whitney cuando tenga que comparar contra CON_PA_MECVV

In [1557]:
stat, p_value = shapiro(SIN_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.6294
Valor p = 0.00000000
El número de filas no sigue una distribución normal (rechazar H0).


In [1561]:
stat, p_value = shapiro(SIN_PA['LRTI_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5158
Valor p = 0.00000000
El número de filas no sigue una distribución normal (rechazar H0).


In [1562]:
stat, p_value = shapiro(SIN_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.5916
Valor p = 0.00000000
El número de filas no sigue una distribución normal (rechazar H0).


In [1563]:
stat, p_value = shapiro(SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 1.0000
Valor p = 1.00000000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Usaré en todos los casos de comparacion el test Mann-Whitney ya que menos la ultima columna (que contiene todo 0), el resto no siguen una distribucion normal 

##### Infecciones respiratorias principales 

In [1564]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0, CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.6832089509594992


In [1565]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0, SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.9113557832989896


In [1566]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.6896369959432476


##### LRTI

In [1567]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['LRTI_diagnosticada'] == 1.0, CON_PA_MECVV['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.9109318987037512


In [1568]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['LRTI_diagnosticada'] == 1.0, SIN_PA['LRTI_diagnosticada'] == 1.0,alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.6863825106164784


In [1569]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['LRTI_diagnosticada'] == 1.0, SIN_PA['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.8770107151211659


##### Exacerbaciones de COPD

In [1572]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['COPD_exacerbations_diagnosticada'] == 1.0, CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 1.0


In [1574]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['COPD_exacerbations_diagnosticada'] == 1.0, SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.8132303236327403


In [1575]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.8809554446418177


##### Fibrosis pulmonar y fibrotorax

In [1576]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 1.0


In [1579]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 1.0


In [1578]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 1.0


## Resultados de laboratorio

Las columnas generadas de estos resultados de laboratorio contienen valores de tipo 'object' y hay que pasarlos previamente a 'float' para poder realizar las operaciones correspondientes.

### Albúmina

In [1432]:
CON_PA.loc[:, 'albumina'] = pd.to_numeric(CON_PA['albumina'], errors='coerce')

promedio_alb = CON_PA['albumina'].mean()
desviacion_estandar_alb = CON_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Promedio ± Desviación Estándar de la albumina:", resultado)

Promedio ± Desviación Estándar de la albumina: 3.34 ± 0.38


In [1433]:
CON_PA_MECVV.loc[:, 'albumina'] = pd.to_numeric(CON_PA_MECVV['albumina'], errors='coerce')

promedio_alb = CON_PA_MECVV['albumina'].mean()
desviacion_estandar_alb = CON_PA_MECVV['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Promedio ± Desviación Estándar de la albumina:", resultado)

Promedio ± Desviación Estándar de la albumina: 3.47 ± 0.55


In [1434]:
SIN_PA.loc[:, 'albumina'] = pd.to_numeric(SIN_PA['albumina'], errors='coerce')

promedio_alb = SIN_PA['albumina'].mean()
desviacion_estandar_alb = SIN_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Promedio ± Desviación Estándar de la albumina:", resultado)

Promedio ± Desviación Estándar de la albumina: 3.58 ± 0.64


#### Pvalor

##### Normalidad

In [1587]:
albpa = CON_PA['albumina'].dropna()
stat, p_value = shapiro(albpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9311
Valor p = 0.56068160
El número de filas sigue una distribución normal (aceptar H0).


In [1584]:
albconpamecvv = CON_PA_MECVV['albumina'].dropna()
stat, p_value = shapiro(albconpamecvv)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9973
Valor p = 0.89985028
El número de filas sigue una distribución normal (aceptar H0).


In [1585]:
albsinpa= SIN_PA['albumina'].dropna()
stat, p_value = shapiro(albsinpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9778
Valor p = 0.92442182
El número de filas sigue una distribución normal (aceptar H0).


Todos los grupos siguen una distribucion normal, puedo usar el T-test

In [1590]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['albumina'], CON_PA_MECVV['albumina'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1591]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['albumina'], SIN_PA['albumina'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1437]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['albumina'], SIN_PA['albumina'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Proteïnas totales

In [1438]:
CON_PA.loc[:, 'proteinas totales'] = pd.to_numeric(CON_PA['proteinas totales'], errors='coerce')

promedio_prot = CON_PA['proteinas totales'].mean()
desviacion_estandar_prot = CON_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Promedio ± Desviación Estándar de las proteinas totales:", resultado)

Promedio ± Desviación Estándar de las proteinas totales: 6.03 ± 0.73


In [1439]:
CON_PA_MECVV.loc[:, 'proteinas totales'] = pd.to_numeric(CON_PA_MECVV['proteinas totales'], errors='coerce')

promedio_prot = CON_PA_MECVV['proteinas totales'].mean()
desviacion_estandar_prot = CON_PA_MECVV['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Promedio ± Desviación Estándar de las proteinas totales:", resultado)

Promedio ± Desviación Estándar de las proteinas totales: 6.47 ± 0.76


In [1440]:
SIN_PA.loc[:, 'proteinas totales'] = pd.to_numeric(SIN_PA['proteinas totales'], errors='coerce')

promedio_prot = SIN_PA['proteinas totales'].mean()
desviacion_estandar_prot = SIN_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Promedio ± Desviación Estándar de las proteinas totales:", resultado)

Promedio ± Desviación Estándar de las proteinas totales: 6.12 ± 0.72


#### Pvalor

##### Normalidad

In [1592]:
protconpa=CON_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protconpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9780
Valor p = 0.95346065
El número de filas sigue una distribución normal (aceptar H0).


In [1593]:
protconpamecvv=CON_PA_MECVV['proteinas totales'].dropna()
stat, p_value = shapiro(protconpamecvv)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9975
Valor p = 0.99159177
El número de filas sigue una distribución normal (aceptar H0).


In [1594]:
protsinpa=SIN_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protsinpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9753
Valor p = 0.86021925
El número de filas sigue una distribución normal (aceptar H0).


In [1441]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['proteinas totales'], CON_PA_MECVV['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1596]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['proteinas totales'], SIN_PA['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1443]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['proteinas totales'], SIN_PA['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Hemoglobina

In [1444]:
CON_PA.loc[:, 'Hb'] = pd.to_numeric(CON_PA['Hb'], errors='coerce')

promedio_hb = CON_PA['Hb'].mean()
desviacion_estandar_hb = CON_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Promedio ± Desviación Estándar de la hemoglobina:", resultado)

Promedio ± Desviación Estándar de la hemoglobina: 12.34 ± 1.77


In [1445]:
CON_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(CON_PA_MECVV['Hb'], errors='coerce')

promedio_hb = CON_PA_MECVV['Hb'].mean()
desviacion_estandar_hb = CON_PA_MECVV['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Promedio ± Desviación Estándar de la hemoglobina:", resultado)

Promedio ± Desviación Estándar de la hemoglobina: 13.22 ± 1.40


In [1446]:
SIN_PA.loc[:, 'Hb'] = pd.to_numeric(SIN_PA['Hb'], errors='coerce')

promedio_hb = SIN_PA['Hb'].mean()
desviacion_estandar_hb = SIN_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Promedio ± Desviación Estándar de la hemoglobina:", resultado)

Promedio ± Desviación Estándar de la hemoglobina: 13.22 ± 2.16


##### Pvalor

##### Normalidad

In [1597]:
hbconpa=CON_PA['Hb'].dropna()
stat, p_value = shapiro(hbconpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9304
Valor p = 0.48501101
El número de filas sigue una distribución normal (aceptar H0).


In [1598]:
hbconpamecvv=CON_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbconpamecvv)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8592
Valor p = 0.25743995
El número de filas sigue una distribución normal (aceptar H0).


In [1599]:
hbsinpa=SIN_PA['Hb'].dropna()
stat, p_value = shapiro(hbsinpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9551
Valor p = 0.37239632
El número de filas sigue una distribución normal (aceptar H0).


In [1447]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Hb'], CON_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1448]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Hb'], SIN_PA['Hb'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1449]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Hb'], SIN_PA['Hb'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Colesterol 

In [1450]:
CON_PA.loc[:, 'colesterol total'] = pd.to_numeric(CON_PA['colesterol total'], errors='coerce')

promedio_col = CON_PA['colesterol total'].mean()
desviacion_estandar_col = CON_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Promedio ± Desviación Estándar del colesterol total:", resultado)

Promedio ± Desviación Estándar del colesterol total: 132.10 ± 34.83


In [1451]:
CON_PA_MECVV.loc[:, 'colesterol total'] = pd.to_numeric(CON_PA_MECVV['colesterol total'], errors='coerce')

promedio_col = CON_PA_MECVV['colesterol total'].mean()
desviacion_estandar_col = CON_PA_MECVV['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Promedio ± Desviación Estándar del colesterol total:", resultado)

Promedio ± Desviación Estándar del colesterol total: 111.90 ± 28.37


In [1452]:
SIN_PA.loc[:, 'colesterol total'] = pd.to_numeric(SIN_PA['colesterol total'], errors='coerce')

promedio_col = SIN_PA['colesterol total'].mean()
desviacion_estandar_col = SIN_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Promedio ± Desviación Estándar del colesterol total:", resultado)

Promedio ± Desviación Estándar del colesterol total: 145.27 ± 40.45


#### Pvalor

##### Normalidad 



In [1600]:
colconpa=CON_PA['colesterol total'].dropna()
stat, p_value = shapiro(colconpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9887
Valor p = 0.99048226
El número de filas sigue una distribución normal (aceptar H0).


In [1601]:
colconpamecvv=CON_PA_MECVV['colesterol total'].dropna()
stat, p_value = shapiro(colconpamecvv)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9462
Valor p = 0.55290474
El número de filas sigue una distribución normal (aceptar H0).


In [1602]:
colsinpa=SIN_PA['colesterol total'].dropna()
stat, p_value = shapiro(colsinpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.9725
Valor p = 0.84373822
El número de filas sigue una distribución normal (aceptar H0).


Usaré tambien T-test ya que todos siguen una distribucion normal 

In [1453]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['colesterol total'], CON_PA_MECVV['colesterol total'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1454]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['colesterol total'], SIN_PA['colesterol total'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1455]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['colesterol total'], SIN_PA['colesterol total'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Leucócitos

In [1456]:
CON_PA.loc[:, 'leucos'] = pd.to_numeric(CON_PA['leucos'], errors='coerce')

promedio_leu = CON_PA['leucos'].mean()
desviacion_estandar_leu = CON_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Promedio ± Desviación Estándar de los leucocitos:", resultado)

Promedio ± Desviación Estándar de los leucocitos: 12.69 ± 4.88


In [1457]:
CON_PA_MECVV.loc[:, 'leucos'] = pd.to_numeric(CON_PA_MECVV['leucos'], errors='coerce')

promedio_leu = CON_PA_MECVV['leucos'].mean()
desviacion_estandar_leu = CON_PA_MECVV['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Promedio ± Desviación Estándar de los leucocitos:", resultado)

Promedio ± Desviación Estándar de los leucocitos: 13.08 ± 6.83


In [1458]:
SIN_PA.loc[:, 'leucos'] = pd.to_numeric(SIN_PA['leucos'], errors='coerce')

promedio_leu = SIN_PA['leucos'].mean()
desviacion_estandar_leu = SIN_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Promedio ± Desviación Estándar de los leucocitos:", resultado)

Promedio ± Desviación Estándar de los leucocitos: 11.52 ± 6.52


#### Pvalor 

##### Normalidad

In [1604]:
leuconpa=CON_PA['leucos'].dropna()
stat, p_value = shapiro(leuconpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8482
Valor p = 0.09127190
El número de filas sigue una distribución normal (aceptar H0).


In [1605]:
leuconpamecvv=CON_PA_MECVV['leucos'].dropna()
stat, p_value = shapiro(leuconpamecvv)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8939
Valor p = 0.40148390
El número de filas sigue una distribución normal (aceptar H0).


In [1606]:
leusinpa=SIN_PA['leucos'].dropna()
stat, p_value = shapiro(leusinpa)
print(f"Estadístico W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El número de filas no sigue una distribución normal (rechazar H0).")
else:
    print("El número de filas sigue una distribución normal (aceptar H0).")

Estadístico W = 0.8178
Valor p = 0.00096236
El número de filas no sigue una distribución normal (rechazar H0).


Usaré T-test menos cuando esté comparando contra el 3r grupo (SIN_PA) donde usaré el test Mann-Whitney ya que es el unico que no sigue una distribución normal

In [1459]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['leucos'], CON_PA_MECVV['leucos'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1460]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['leucos'], SIN_PA['leucos'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1461]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['leucos'], SIN_PA['leucos'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Linfocitos %

In [1462]:
CON_PA.loc[:, 'limfos'] = pd.to_numeric(CON_PA['limfos'], errors='coerce')

promedio_limf = CON_PA['limfos'].mean()
desviacion_estandar_limf = CON_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Promedio ± Desviación Estándar de los limfocitos %:", resultado)

Promedio ± Desviación Estándar de los limfocitos %: 16.70 ± 12.29


In [1463]:
CON_PA_MECVV.loc[:, 'limfos'] = pd.to_numeric(CON_PA_MECVV['limfos'], errors='coerce')

promedio_limf = CON_PA_MECVV['limfos'].mean()
desviacion_estandar_limf = CON_PA_MECVV['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Promedio ± Desviación Estándar de los limfocitos %:", resultado)

Promedio ± Desviación Estándar de los limfocitos %: 17.20 ± 9.35


In [1464]:
SIN_PA.loc[:, 'limfos'] = pd.to_numeric(SIN_PA['limfos'], errors='coerce')

promedio_limf = SIN_PA['limfos'].mean()
desviacion_estandar_limf = SIN_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Promedio ± Desviación Estándar de los limfocitos %:", resultado)

Promedio ± Desviación Estándar de los limfocitos %: 19.50 ± 11.57


#### Pvalor

In [1465]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['limfos'], CON_PA_MECVV['limfos'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1466]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['limfos'], SIN_PA['limfos'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1467]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['limfos'], SIN_PA['limfos'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Proteina C Reactiva

In [1468]:
CON_PA.loc[:, 'prot C react'] = pd.to_numeric(CON_PA['prot C react'], errors='coerce')

promedio_protcr = CON_PA['prot C react'].mean()
desviacion_estandar_protcr = CON_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Promedio ± Desviación Estándar de la proteina C reactiva:", resultado)

Promedio ± Desviación Estándar de la proteina C reactiva: 8.90 ± 5.47


In [1469]:
CON_PA_MECVV.loc[:, 'prot C react'] = pd.to_numeric(CON_PA_MECVV['prot C react'], errors='coerce')

promedio_protcr = CON_PA_MECVV['prot C react'].mean()
desviacion_estandar_protcr = CON_PA_MECVV['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Promedio ± Desviación Estándar de la proteina C reactiva:", resultado)

Promedio ± Desviación Estándar de la proteina C reactiva: 8.88 ± 3.63


In [1470]:
SIN_PA.loc[:, 'prot C react'] = pd.to_numeric(SIN_PA['prot C react'], errors='coerce')

promedio_protcr = SIN_PA['prot C react'].mean()
desviacion_estandar_protcr = SIN_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Promedio ± Desviación Estándar de la proteina C reactiva:", resultado)

Promedio ± Desviación Estándar de la proteina C reactiva: 6.16 ± 6.19


#### Pvalor

In [1471]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['prot C react'], CON_PA_MECVV['prot C react'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1472]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['prot C react'], SIN_PA['prot C react'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1473]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['prot C react'], SIN_PA['prot C react'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Urea 

In [1474]:
CON_PA.loc[:, 'urea'] = pd.to_numeric(CON_PA['urea'], errors='coerce')

promedio_urea = CON_PA['urea'].mean()
desviacion_estandar_urea = CON_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Promedio ± Desviación Estándar de la urea:", resultado)

Promedio ± Desviación Estándar de la urea: 62.17 ± 36.37


In [1475]:
CON_PA_MECVV.loc[:, 'urea'] = pd.to_numeric(CON_PA_MECVV['urea'], errors='coerce')

promedio_urea = CON_PA_MECVV['urea'].mean()
desviacion_estandar_urea = CON_PA_MECVV['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Promedio ± Desviación Estándar de la urea:", resultado)

Promedio ± Desviación Estándar de la urea: 62.37 ± 37.72


In [1476]:
SIN_PA.loc[:, 'urea'] = pd.to_numeric(SIN_PA['urea'], errors='coerce')

promedio_urea = SIN_PA['urea'].mean()
desviacion_estandar_urea = SIN_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Promedio ± Desviación Estándar de la urea:", resultado)

Promedio ± Desviación Estándar de la urea: 55.03 ± 30.66


#### Pvalor

In [1477]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['urea'], CON_PA_MECVV['urea'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1478]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['urea'], SIN_PA['urea'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1479]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['urea'], SIN_PA['urea'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Filtrado glomerular CDK-EPI

In [1480]:
CON_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(CON_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = CON_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = CON_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI: 56.00 ± 12.65


In [1481]:
CON_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(CON_PA_MECVV['FGE CDK-EPI'], errors='coerce')

promedio_fg = CON_PA_MECVV['FGE CDK-EPI'].mean()
desviacion_estandar_fg = CON_PA_MECVV['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI: 54.67 ± 13.32


In [1482]:
SIN_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SIN_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = SIN_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = SIN_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI: 58.56 ± 18.40


#### Pvalor

In [1483]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['FGE CDK-EPI'], CON_PA_MECVV['FGE CDK-EPI'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1484]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['FGE CDK-EPI'], SIN_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1485]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['FGE CDK-EPI'], SIN_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Filtrado glomerular MDRD

In [1486]:
CON_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(CON_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = CON_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = CON_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular MDRD:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular MDRD: 51.50 ± 0.71


In [1487]:
CON_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(CON_PA_MECVV['FGE MDRD'], errors='coerce')

promedio_fgmd = CON_PA_MECVV['FGE MDRD'].mean()
desviacion_estandar_fgmd = CON_PA_MECVV['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular MDRD:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular MDRD: nan ± nan


In [1488]:
SIN_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SIN_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = SIN_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = SIN_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular MDRD:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular MDRD: 37.67 ± 17.43


#### Pvalor

In [1489]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['FGE MDRD'], CON_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1490]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['FGE MDRD'], SIN_PA['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1491]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['FGE MDRD'], SIN_PA['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA_MECVV y CON_PA_MECVV:", p_val_23)

P-valor entre CON_PA_MECVV y CON_PA_MECVV: nan
